# Chapter 12: Custom Models and Training with TensorFlow

**Tujuan:** Menguasai API TensorFlow rendah-level: tensor, variable, operasi, custom layers/models, loss, metric, gradient tape, dan `@tf.function`.

---

## 1. Quick Tour of TensorFlow

- **Tensors**: mirip `ndarray`, tapi dapat GPU/TPU  
- **Operations**: `tf.add`, `tf.matmul`, dll.  
- **Variables**: stateful tensors yang dapat di‑update

In [1]:
import tensorflow as tf

# Tensor vs NumPy
t1 = tf.constant([[1,2],[3,4]])
n1 = t1.numpy()  # konversi ke NumPy
print("Tensor:", t1, "NumPy:", n1)

# Variable & update
v = tf.Variable(2.0)
print("Before:", v.numpy())
v.assign_add(3.0)
print("After:", v.numpy())

Tensor: tf.Tensor(
[[1 2]
 [3 4]], shape=(2, 2), dtype=int32) NumPy: [[1 2]
 [3 4]]
Before: 2.0
After: 5.0


## 2. Custom Layers & Models

### 2.1 Custom Layer

In [2]:
from tensorflow.keras import layers

class MyDense(layers.Layer):
    def __init__(self, units, **kwargs):
        super().__init__(**kwargs)
        self.units = units

    def build(self, input_shape):
        # buat variable bobot dan bias
        self.w = self.add_weight(
            shape=(input_shape[-1], self.units),
            initializer="random_normal",
            trainable=True, name="w"
        )
        self.b = self.add_weight(
            shape=(self.units,), initializer="zeros",
            trainable=True, name="b"
        )

    def call(self, inputs):
        return tf.matmul(inputs, self.w) + self.b

# Demo: pakai custom layer
layer = MyDense(4)
x = tf.random.normal((2,3))
print("Output MyDense:", layer(x))

Output MyDense: tf.Tensor(
[[ 0.0474494  -0.04540867 -0.0068359   0.02124174]
 [ 0.00522392 -0.04822998 -0.00921627  0.01341575]], shape=(2, 4), dtype=float32)


### 2.2 Custom Model

In [3]:
from tensorflow.keras import Model, Input

class MyMLP(Model):
    def __init__(self):
        super().__init__()
        self.d1 = MyDense(16)
        self.act = layers.ReLU()
        self.d2 = MyDense(1)

    def call(self, inputs):
        x = self.d1(inputs)
        x = self.act(x)
        return self.d2(x)

# Instansiasi dan panggil
model = MyMLP()
y = model(tf.random.normal((5,10)))
print("MyMLP output shape:", y.shape)

MyMLP output shape: (5, 1)


## 3. Custom Loss & Metric

In [4]:
# Custom loss: Huber loss
def huber_loss(y_true, y_pred, delta=1.0):
    err = y_true - y_pred
    cond = tf.abs(err) <= delta
    squared = 0.5 * tf.square(err)
    linear  = delta * (tf.abs(err) - 0.5*delta)
    return tf.where(cond, squared, linear)

# Custom metric: mean absolute percentage error
class MAPE(tf.keras.metrics.Metric):
    def __init__(self, name="mape", **kwargs):
        super().__init__(name=name, **kwargs)
        self.total = self.add_weight(name="total", initializer="zeros")
        self.count = self.add_weight(name="count", initializer="zeros")

    def update_state(self, y_true, y_pred, sample_weight=None):
        mape = tf.reduce_mean(tf.abs((y_true - y_pred) / y_true))
        self.total.assign_add(mape)
        self.count.assign_add(1.0)

    def result(self):
        return self.total / self.count

    def reset_states(self):
        self.total.assign(0.0)
        self.count.assign(0.0)

## 4. Custom Training Loop (GradientTape)

In [5]:
# Dataset sintetis: y = 3x + 2 + noise
import numpy as np
X = np.random.rand(1000,1).astype("float32")
y = 3*X + 2 + 0.1*np.random.randn(1000,1).astype("float32")

# Buat tf.data.Dataset
ds = tf.data.Dataset.from_tensor_slices((X,y)).shuffle(1000).batch(32)

# Model sederhana
linear = MyDense(1)
optimizer = tf.keras.optimizers.SGD(learning_rate=0.1)

# Training loop
for epoch in range(5):
    for x_batch, y_batch in ds:
        with tf.GradientTape() as tape:
            preds = linear(x_batch)
            loss = tf.reduce_mean(tf.square(y_batch - preds))
        grads = tape.gradient(loss, linear.trainable_variables)
        optimizer.apply_gradients(zip(grads, linear.trainable_variables))
    print(f"Epoch {epoch+1}: loss = {loss.numpy():.4f}")

print("Learned parameters:", linear.w.numpy().flatten(), linear.b.numpy())

Epoch 1: loss = 0.0551
Epoch 2: loss = 0.0631
Epoch 3: loss = 0.0177
Epoch 4: loss = 0.0139
Epoch 5: loss = 0.0078
Learned parameters: [2.8116302] [2.0936534]


## 5. Accelerate with `@tf.function`

In [6]:
@tf.function
def train_step(x, y):
    with tf.GradientTape() as tape:
        preds = linear(x)
        loss = tf.reduce_mean(tf.square(y - preds))
    grads = tape.gradient(loss, linear.trainable_variables)
    optimizer.apply_gradients(zip(grads, linear.trainable_variables))
    return loss

# Training loop dengan tf.function
for epoch in range(5):
    for x_batch, y_batch in ds:
        loss = train_step(x_batch, y_batch)
    print(f"[tf.function] Epoch {epoch+1}: loss = {loss.numpy():.4f}")

[tf.function] Epoch 1: loss = 0.0209
[tf.function] Epoch 2: loss = 0.0185
[tf.function] Epoch 3: loss = 0.0131
[tf.function] Epoch 4: loss = 0.0076
[tf.function] Epoch 5: loss = 0.0111


# Ringkasan Chapter 12
- Tensor & Variable adalah blok dasar TF.

- Custom Layer & Model mudah dibuat dengan subclassing.

- Custom Loss/Metric untuk kebutuhan khusus.

- GradientTape memberi kontrol penuh training loop.

- `@tf.function` mempercepat eksekusi dengan tracing.